<a href="https://colab.research.google.com/github/Rahafsjob/Capstone_Project_WeCloudData/blob/main/Stage2/Data_Quality_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality for Capston Project - Stage 2 ⭐
- users
- cohorts
- cohort_enrollments
- cohort_courses
- assiements
- Cohort_Assignment
- cohort_course_enrollments
- Quizzes
- attendanc
- Change_Requests
- Student_Status
- Change_Requests

# Install libraries

In [ ]:
!pip install SQLAlchemy

In [ ]:
!pip install PyMySQL

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.0 MB/s eta 0:00:00


# Import library

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
import numpy as np

# Make Connection

In [ ]:
ENDPOINT = "database-1.cipmeg2oi9nm.us-east-1.rds.amazonaws.com"
PORT = "3306"
USERNAME = "admin"
PASSWORD = "Rahaf123_"
DATABASENAME = "demo_db"

engine = create_engine(f"mysql+pymysql://{USERNAME}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASENAME}")

# Data Quality Dimensions
1. Completeness: Are all required values present?

2. Uniqueness: Are there duplicate rows or duplicate IDs?

3. imeliness: Is the data up-to-date and relevant?

4. Validity: Do values follow the expected format and range?

5. Accuracy: Do values represent real-world entities correctly?

6. Consistency: Are values consistent across records and sources?



# users

In [ ]:
query = f"SELECT * FROM users"
users_df = pd.read_sql(query, engine)
users_df.head()

,id,role,email,client_id,last_name,first_name,student_id,_airbyte_raw_id,_airbyte_extracted_at,_airbyte_meta
0,8705,{student},Frank.Martin@gmail.com,client_1,Martin,Frank,Stud0177,001deb23-f94a-4ad1-89ff-800bb123c73c,2025-05-04 18:49:39.639,"{""changes"": []}"
1,8890,{student},Karen.Ortega@hotmail.com,client_2,Ortega,Karen,Stud0507,00303f47-7916-47e2-a7c1-f08e3f9847d0,2025-05-04 18:49:39.639,"{""changes"": []}"
2,9984,{student},David.Meyer@hotmail.com,client_1,Meyer,David,Stud0877,0031197f-21e3-4c41-974e-fb32b49a48cc,2025-05-04 18:49:39.639,"{""changes"": []}"
3,9145,{student},Heidi.Ball@outlook.com,client_2,Ball,Heidi,Stud0625,005f7c63-98a8-4af1-bbc9-81718fa73d46,2025-05-04 18:49:39.639,"{""changes"": []}"
4,8983,{student},Christine.Parker@gmail.com,client_2,Parker,Christine,Stud0527,01dda834-752d-4e3f-9c2c-0c66cb54bb16,2025-05-04 18:49:39.639,"{""changes"": []}"


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql

# بيانات الاتصال
ENDPOINT = "raw-data.csb002k2ask1.us-east-1.rds.amazonaws.com"
PORT = "3306"
USERNAME = "admin"
PASSWORD = "Shmuokh001*"
DATABASENAME = "raw_data"

# الاتصال
engine = create_engine(f"mysql+pymysql://{USERNAME}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASENAME}")

# قراءة البيانات
df = pd.read_sql("SELECT * FROM attendance", engine)
total_rows = len(df)

# تحويل تاريخ الحضور إلى datetime
if 'attendance_date' in df.columns:
    df['attendance_date'] = pd.to_datetime(df['attendance_date'], errors='coerce')

# حساب معايير الجودة
results = {
    "completeness_pct": df.notnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100,
    "uniqueness_pct": df.drop_duplicates().shape[0] / df.shape[0] * 100,
    "timeliness_pct": df['attendance_date'].ge(pd.Timestamp.today() - pd.Timedelta(days=30)).mean() * 100 if 'attendance_date' in df.columns else None,
    "validity_pct": df['attendance_status'].isin(['P', 'X', 'E', 'L']).mean() * 100 if 'attendance_status' in df.columns else None,
    "accuracy_pct": df['attendance_date'].le(pd.Timestamp.today()).mean() * 100 if 'attendance_date' in df.columns else None,
    "consistency_pct": df.groupby(['StudentID', 'Final_Cohort'])['level'].nunique().le(1).mean() * 100 if set(['StudentID', 'Final_Cohort', 'level']).issubset(df.columns) else None
}

# عرض النتائج
results_df = pd.DataFrame([results])
print(results_df)


   completeness_pct  uniqueness_pct  timeliness_pct validity_pct  \
0         99.880814       99.947028        2.786312         None   

   accuracy_pct  consistency_pct  
0         100.0            100.0  


In [ ]:
query = f"SELECT * FROM attendance"
enrollments_df = pd.read_sql(query, engine)
enrollments_df.head()

,StudentID,attendance_date,attendance_data,course_name,cohort_number,filename,Final_Cohort,level
0,Stud0013,2024-08-05,p,NLP and LLM Finetuning,3,Attendance Register - NLP and LLM Finetuning -...,NLP and LLM Finetuning - Cohort 3,3
1,Stud0103,2024-08-05,p,NLP and LLM Finetuning,3,Attendance Register - NLP and LLM Finetuning -...,NLP and LLM Finetuning - Cohort 3,3
2,Stud0173,2024-08-05,x,NLP and LLM Finetuning,3,Attendance Register - NLP and LLM Finetuning -...,NLP and LLM Finetuning - Cohort 3,3
3,Stud0198,2024-08-05,p,NLP and LLM Finetuning,3,Attendance Register - NLP and LLM Finetuning -...,NLP and LLM Finetuning - Cohort 3,3
4,Stud0220,2024-08-05,p,NLP and LLM Finetuning,3,Attendance Register - NLP and LLM Finetuning -...,NLP and LLM Finetuning - Cohort 3,3


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import re

# ⚙️ إعداد الاتصال بقاعدة البيانات
ENDPOINT = "raw-data.csb002k2ask1.us-east-1.rds.amazonaws.com"
PORT = "3306"
USERNAME = "admin"
PASSWORD = "Shmuokh001*"
DATABASENAME = "raw_data"

# إنشاء الاتصال باستخدام SQLAlchemy
engine = create_engine(f"mysql+pymysql://{USERNAME}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASENAME}")

# ✅ تحميل البيانات من الجدول (يرجى تحديد اسم الجدول الصحيح)
TABLE_NAME = "attendance"  # ✏️ عدل الاسم حسب الحاجة
df = pd.read_sql(f"SELECT * FROM {TABLE_NAME}", engine)

# ⚙️ تهيئة تقرير الجودة
dq_report = {}

# ✅ 1. Completeness - نسبة القيم غير المفقودة
total_cells = df.size
non_missing_cells = df.notnull().sum().sum()
completeness_pct = (non_missing_cells / total_cells) * 100
dq_report["Completeness (overall)"] = f"{completeness_pct:.2f}%"

# ✅ 2. Uniqueness - الصفوف المميزة بالكامل
unique_rows = df.drop_duplicates().shape[0]
uniqueness_pct = (unique_rows / len(df)) * 100
dq_report["Uniqueness (all rows)"] = f"{uniqueness_pct:.2f}%"

# ✅ 3. Timeliness - تاريخ الحضور خلال آخر 30 يوم
if 'attendance_date' in df.columns:
    df['attendance_date'] = pd.to_datetime(df['attendance_date'], errors='coerce')
    recent_dates_pct = df['attendance_date'].ge(pd.Timestamp.today() - pd.Timedelta(days=30)).mean() * 100
    dq_report["Timeliness (last 30 days)"] = f"{recent_dates_pct:.2f}%"
else:
    dq_report["Timeliness (last 30 days)"] = "Column 'attendance_date' not found"

# ✅ 4. Validity - حالة الحضور (P, X, E, L)
if 'attendance_data' in df.columns:
    valid_status = df['attendance_data'].isin(['P', 'X', 'E', 'L'])
    validity_pct = valid_status.mean() * 100
    dq_report["Validity (attendance_data)"] = f"{validity_pct:.2f}%"
else:
    dq_report["Validity (attendance_data)"] = "Column 'attendance_data' not found"

# ✅ 5. Accuracy - أن يكون التاريخ ≤ اليوم الحالي
if 'attendance_date' in df.columns:
    accurate_dates_pct = df['attendance_date'].le(pd.Timestamp.today()).mean() * 100
    dq_report["Accuracy (attendance_date <= today)"] = f"{accurate_dates_pct:.2f}%"

# ✅ 6. Consistency - نفس StudentID مع Final_Cohort له مستوى واحد فقط
if set(['StudentID', 'Final_Cohort', 'level']).issubset(df.columns):
    consistency_check = df.groupby(['StudentID', 'Final_Cohort'])['level'].nunique().le(1)
    consistency_pct = consistency_check.mean() * 100
    dq_report["Consistency (level by StudentID & Cohort)"] = f"{consistency_pct:.2f}%"
else:
    dq_report["Consistency (level by StudentID & Cohort)"] = "Required columns not found"

# ✅ طباعة تقرير الجودة
print("\n📊 Data Quality Report (attendance):")
for dimension, result in dq_report.items():
    print(f"- {dimension}: {result}")

# ✅ تحويل التقرير إلى DataFrame وحفظه
data_quality_report = pd.DataFrame(
    list(dq_report.items()),
    columns=["Dimension", "Percentage"]
)

data_quality_report.to_csv('attendance_data_quality_report.csv', index=False)
print("\n📁 ✅ تم حفظ التقرير في ملف: attendance_data_quality_report.csv")



📊 Data Quality Report (attendance):
- Completeness (overall): 99.88%
- Uniqueness (all rows): 99.95%
- Timeliness (last 30 days): 2.79%
- Validity (attendance_data): 23.56%
- Accuracy (attendance_date <= today): 100.00%
- Consistency (level by StudentID & Cohort): 100.00%

📁 ✅ تم حفظ التقرير في ملف: attendance_data_quality_report.csv


In [ ]:
results_df.to_csv('attendance_report.csv', index=False)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql

# 🛠️ معلومات الاتصال بقاعدة البيانات
ENDPOINT = "raw-data.csb002k2ask1.us-east-1.rds.amazonaws.com"
PORT = "3306"
USERNAME = "admin"
PASSWORD = "Shmuokh001*"
DATABASENAME = "raw_data"

# ⚙️ إنشاء الاتصال
engine = create_engine(f"mysql+pymysql://{USERNAME}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASENAME}")

# 📥 قراءة البيانات من جدول attendance
df = pd.read_sql("SELECT * FROM attendance", engine)

# 🕒 تحويل تاريخ الحضور إلى datetime إن وجد
if 'attendance_date' in df.columns:
    df['attendance_date'] = pd.to_datetime(df['attendance_date'], errors='coerce')

# ✅ 1. Completeness
missing_summary = df.isnull().sum()
completeness_result = "Warning ⚠️" if missing_summary.sum() > 0 else "Pass ✅"
completeness_details = f"Missing values found in {missing_summary[missing_summary > 0].count()} columns"

# ✅ 2. Uniqueness
duplicate_rows = df.duplicated().sum()
duplicate_ids = df['StudentID'].duplicated().sum() if 'StudentID' in df.columns else 0
uniqueness_result = "Fail ❌" if duplicate_ids > 0 else "Pass ✅"
uniqueness_details = f"Duplicate rows: {duplicate_rows}, Duplicate Student IDs: {duplicate_ids}"

# ✅ 3. Consistency
if set(['StudentID', 'Final_Cohort', 'level']).issubset(df.columns):
    level_consistent = df.groupby(['StudentID', 'Final_Cohort'])['level'].nunique().le(1).all()
    consistency_result = "Pass ✅" if level_consistent else "Fail ❌"
    consistency_details = f"Level consistency within StudentID/Cohort: {level_consistent}"
else:
    consistency_result = "N/A"
    consistency_details = "Required columns for consistency not found"

# ✅ 4. Timeliness
current_date = pd.Timestamp.now()
if 'attendance_date' in df.columns:
    recent_data_pct = df['attendance_date'].dropna().ge(current_date - pd.Timedelta(days=30)).mean() * 100
    future_dates_count = df['attendance_date'].dropna().gt(current_date).sum()
    timeliness_result = "Warning ⚠️" if recent_data_pct < 100 else "Pass ✅"
    timeliness_details = f"{recent_data_pct:.1f}% of records from last 30 days"
else:
    timeliness_result = "N/A"
    timeliness_details = "Column 'attendance_date' not found"

# ✅ 5. Accuracy (no future dates)
if 'attendance_date' in df.columns:
    accuracy_result = "Fail ❌" if future_dates_count > 0 else "Pass ✅"
    accuracy_details = f"Found {future_dates_count} future dates"
else:
    accuracy_result = "N/A"
    accuracy_details = "Column 'attendance_date' not found"

# ✅ 6. Validity (attendance_status values)
if 'attendance_data' in df.columns:
    valid_statuses = ['P', 'X', 'E', 'L']
    invalid_statuses_count = (~df['attendance_data'].isin(valid_statuses)).sum()
    validity_result = "Fail ❌" if invalid_statuses_count > 0 else "Pass ✅"
    validity_details = f"{invalid_statuses_count} invalid values in 'attendance_data'"
else:
    validity_result = "N/A"
    validity_details = "Column 'attendance_status' not found"

# 📊 إنشاء تقرير جودة البيانات
attendance_quality_report = pd.DataFrame({
    "Dimension": [
        "Completeness",
        "Uniqueness",
        "Consistency",
        "Timeliness",
        "Validity"
    ],
    "Result": [
        completeness_result,
        uniqueness_result,
        consistency_result,
        timeliness_result,
        validity_result
    ],
    "Details": [
        completeness_details,
        uniqueness_details,
        consistency_details,
        timeliness_details,
        validity_details
    ]
})

# 📋 طباعة التقرير
print(f"\nTotal rows analyzed: {len(df)}")
print("\n📊 Data Quality Report:")
print(attendance_quality_report)

# 📈 توزيع حالات الحضور إن وجد
if 'attendance_status' in df.columns:
    print("\n📌 Attendance Status Distribution (%):")
    print(df['attendance_status'].value_counts(normalize=True) * 100)



Total rows analyzed: 9439

📊 Data Quality Report:
      Dimension      Result                                          Details
0  Completeness  Warning ⚠️                Missing values found in 1 columns
1    Uniqueness      Fail ❌   Duplicate rows: 5, Duplicate Student IDs: 9018
2   Consistency      Pass ✅  Level consistency within StudentID/Cohort: True
3    Timeliness  Warning ⚠️                2.8% of records from last 30 days
4      Validity      Fail ❌         7215 invalid values in 'attendance_data'


In [ ]:
attendance_quality_report

,Dimension,Result,Details
0,Completeness,Warning ⚠️,Missing values found in 1 columns
1,Uniqueness,Fail ❌,"Duplicate rows: 5, Duplicate Student IDs: 9018"
2,Consistency,Pass ✅,Level consistency within StudentID/Cohort: True
3,Timeliness,Warning ⚠️,2.8% of records from last 30 days
4,Validity,Fail ❌,7215 invalid values in 'attendance_data'


In [ ]:
attendance_quality_report.to_csv('attendance_quality_report.csv', index=False)

In [ ]:
# Completeness
missing_summary = users_df.isnull().sum()
print("Missing values per column:\n", missing_summary)

Missing values per column:
 id                       0
role                     0
email                    0
client_id                0
last_name                0
first_name               0
student_id               0
_airbyte_raw_id          0
_airbyte_extracted_at    0
_airbyte_meta            0
dtype: int64


In [ ]:
# Uniqueness
duplicates = users_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Duplicate IDs (if there's an 'id' column)
if 'id' in users_df.columns:
    duplicate_ids = users_df['id'].duplicated().sum()
    print(f"Number of duplicate IDs: {duplicate_ids}")

Number of duplicate rows: 0
Number of duplicate IDs: 1876


In [ ]:
# Validity: Check if fields match expected formats.
import re

pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
valid_emails = users_df['email'].str.match(pattern)
print(f"Invalid emails: {len(users_df[~valid_emails])}")

Invalid emails: 0


In [ ]:
# Validity: Check if fields match expected formats.
student_pattern = r'^Stud\d{4}$'
valid_ids = users_df['student_id'].str.match(student_pattern)
print(f"Invalid student IDs: {len(users_df[~valid_ids])}")

Invalid student IDs: 0


In [ ]:
print("Unique client IDs:", users_df['client_id'].unique())

Unique client IDs: ['client_1' 'client_2']


In [ ]:
# Consistency: Check if fields like role have consistent formatting:
print("Unique roles:", users_df['role'].unique())

Unique roles: ['{student}' '{student,admin}' '{student,admin,admin}' '{admin,admin}']


In [ ]:
data_quality_report = pd.DataFrame({
    "Dimension": [
        "Completeness",
        "Uniqueness",
        "Validity",
        "Accuracy",
        "Consistency"
    ],
    "Result": [
        "Pass ✅",
        "Fail ❌",  # because of 1876 duplicate IDs
        "Pass ✅",
        "Issue ⚠️",
        "Fail ❌"   # because of inconsistent role values
    ],
    "Details": [
        "No nulls detected",
        "1876 duplicate IDs found",
        "Emails and student_ids match expected formats",
        "Client and name accuracy needs verification",
        "Roles contain inconsistent and repeated values"
    ]
})

data_quality_report


,Dimension,Result,Details
0,Completeness,Pass ✅,No nulls detected
1,Uniqueness,Fail ❌,1876 duplicate IDs found
2,Validity,Pass ✅,Emails and student_ids match expected formats
3,Accuracy,Issue ⚠️,Client and name accuracy needs verification
4,Consistency,Fail ❌,Roles contain inconsistent and repeated values


In [ ]:
data_quality_report.to_csv('users_data_quality_report2.csv', index=False)

# cohorts

In [ ]:
query = f"SELECT * FROM cohorts"
cohorts_df=pd.read_sql(query, engine)
cohorts_df.head()

,id,days,name,times,end_date,client_id,created_at,program_id,start_date,resource_id,cohort_enrollments_count,_airbyte_raw_id,_airbyte_extracted_at,_airbyte_meta
0,245,{},M2.3-GenAI-Intro-B2,{},None,client_1,2024-01-26 18:35:02.972686,37,None,None,28,0025c46b-1e74-4c3e-8b86-c0779cf020b1,2025-05-04 18:42:55.806,"{""changes"": []}"
1,279,{},M2.3-GenAI-Intro-b7,{},None,client_1,2024-03-28 09:19:03.775730,37,None,None,44,010ed965-6c49-40ab-b827-dc647a2408f2,2025-05-04 18:49:38.373,"{""changes"": []}"
2,288,"{Mon,Tue,Wed,Thu,Fri}",M2.2-ML-B3,{},None,client_1,2024-04-30 11:32:05.733903,37,None,None,17,01318df0-acde-45f5-952f-33eb16b783a3,2025-05-04 18:49:38.373,"{""changes"": []}"
3,251,{},Data Engineer -Feb - Online 1,{},None,client_2,2024-02-04 06:40:31.910747,41,None,None,59,01444ab5-1b15-4786-80ad-454385377428,2025-05-04 18:42:55.806,"{""changes"": []}"
4,283,{},M2.4-Finetuning-B6,{},None,client_1,2024-03-28 09:29:11.248108,37,None,None,33,033e1f45-8315-4d4b-b386-a78fe5686b01,2025-05-04 18:42:55.806,"{""changes"": []}"


In [ ]:
# Completeness
missing_summary = cohorts_df.isnull().sum()
print("Missing values per column:\n", missing_summary)

# Uniqueness
duplicates = cohorts_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Duplicate IDs
duplicate_ids = cohorts_df['id'].duplicated().sum()
print(f"Number of duplicate IDs: {duplicate_ids}")

# Validity checks
# Check if days column contains valid format (empty dict or dict-like)
try:
    days_valid = cohorts_df['days'].apply(lambda x: x == '{}' or isinstance(x, dict) or pd.isna(x)).all()
    print(f"Invalid days format: {len(cohorts_df[~cohorts_df['days'].apply(lambda x: x == '{}' or isinstance(x, dict) or pd.isna(x))])}")
except Exception as e:
    print(f"Days validation error: {str(e)}")
    days_valid = False

# Check if times column is non-empty string
times_valid = cohorts_df['times'].apply(lambda x: isinstance(x, str) and len(x) > 0).all()
print(f"Invalid times values: {len(cohorts_df[~cohorts_df['times'].apply(lambda x: isinstance(x, str) and len(x) > 0)])}")

# Check client_id format
client_id_valid = cohorts_df['client_id'].apply(lambda x: isinstance(x, str) and x.startswith('client_')).all()
print(f"Invalid client IDs: {len(cohorts_df[~cohorts_df['client_id'].apply(lambda x: isinstance(x, str) and x.startswith('client_'))])}")

# Check created_at is valid datetime
try:
    pd.to_datetime(cohorts_df['created_at'])
    datetime_valid = True
except Exception as e:
    datetime_valid = False
    print(f"Invalid datetime values in created_at: {str(e)}")

# Create data quality report
cohorts_quality_report = pd.DataFrame({
    "Dimension": [
        "Completeness",
        "Uniqueness",
        "Validity",
        "Accuracy",
        "Consistency"
    ],
    "Result": [
        "Warning ⚠️" if missing_summary[['end_date', 'start_date']].sum() > 0 else "Pass ✅",
        "Fail ❌" if duplicate_ids > 0 else "Pass ✅",
        "Warning ⚠️" if not all([days_valid, times_valid, client_id_valid, datetime_valid]) else "Pass ✅",
        "Needs Verification 🔍",
        "Warning ⚠️" if cohorts_df['client_id'].nunique() > 1 else "Pass ✅"
    ],
    "Details": [
        f"Missing values in end_date ({missing_summary['end_date']}) and start_date ({missing_summary['start_date']})",
        f"{duplicate_ids} duplicate IDs found (critical issue)" if duplicate_ids > 0 else "No duplicate IDs found",
        f"Validity issues: Days format ({not days_valid}), Times ({not times_valid}), Client IDs ({not client_id_valid}), Dates ({not datetime_valid})",
        "Manual verification needed for cohort names and program mappings",
        f"{cohorts_df['client_id'].nunique()} different client IDs found" if cohorts_df['client_id'].nunique() > 1 else "Consistent client IDs"
    ]
})

cohorts_quality_report

Missing values per column:
 id                            0
days                          0
name                          0
times                         0
end_date                    232
client_id                     0
created_at                    0
program_id                    0
start_date                  232
resource_id                 244
cohort_enrollments_count      0
_airbyte_raw_id               0
_airbyte_extracted_at         0
_airbyte_meta                 0
dtype: int64
Number of duplicate rows: 0
Number of duplicate IDs: 183
Invalid days format: 24
Invalid times values: 0
Invalid client IDs: 0


,Dimension,Result,Details
0,Completeness,Warning ⚠️,Missing values in end_date (232) and start_dat...
1,Uniqueness,Fail ❌,183 duplicate IDs found (critical issue)
2,Validity,Warning ⚠️,"Validity issues: Days format (True), Times (Fa..."
3,Accuracy,Needs Verification 🔍,Manual verification needed for cohort names an...
4,Consistency,Warning ⚠️,2 different client IDs found


In [ ]:
cohorts_quality_report.to_csv('cohorts_quality_report.csv', index=False)

Breakdown of the Validity Status:
Days format (True)

✅ Valid: The days column passed its validation check (all values are either empty dicts {}, proper dictionaries, or null)

This means the format/structure of data in this column is correct

Times (False)

❌ Invalid: The times column failed its validation check

This means some values either:

Aren't strings

Are empty strings

Don't match expected time format

Client IDs (False)

❌ Invalid: The client_id column failed its validation check

Some values don't start with 'client_' as expected

May contain:

Null values

Different prefix

Malformed entries

Dates (False)

❌ Invalid: The date-related columns failed validation

Likely issues with:

created_at contains non-datetime values

end_date/start_date have invalid formats

Unexpected null values where not allowed

# cohort enrollments

In [ ]:
query = f"SELECT * FROM cohort_enrollments"
enrollments_df = pd.read_sql(query, engine)
enrollments_df.head()

,id,user_id,client_id,cohort_id,created_at,portfolio_published,_airbyte_raw_id,_airbyte_extracted_at,_airbyte_meta
0,15695,8680,client_1,324,2025-03-27 13:29:34.691648,0,0063619b-8614-4ed7-b656-5ef6db81dd55,2025-05-04 18:49:38.915,"{""changes"": []}"
1,12501,9119,client_1,269,2024-03-08 07:32:32.940634,0,00a1fbee-9238-4263-adf8-1d7e1c68e14d,2025-05-04 18:49:38.915,"{""changes"": []}"
2,15519,10236,client_2,248,2025-02-16 12:31:20.679569,0,00b6b198-799e-46ee-bacb-388317768021,2025-05-04 18:49:38.915,"{""changes"": []}"
3,15576,9533,client_1,323,2025-02-19 08:36:46.737145,0,00b7bad1-7b70-4ea3-97e3-89d6c1887c13,2025-05-04 18:49:38.915,"{""changes"": []}"
4,13436,9461,client_1,297,2024-06-14 12:37:38.869481,0,00bfa952-f747-4437-8a96-f5f5d693834c,2025-05-04 18:49:38.915,"{""changes"": []}"


# assiemnets

In [ ]:
query = f"select * from assiements"
assignments_df = pd.read_sql(query, engine)
assignments_df.head()

,Student_ID,week1,week2,week3,week4,week5,week6,week7,final_score,name
0,Stud0449,100%,100%,100%,100%,100%,100%,100%,100.0,Data Governance (Online Batch)
1,Stud0450,100%,100%,100%,100%,100%,100%,100%,100.0,Data Governance (Online Batch)
2,Stud0451,100%,100%,100%,100%,100%,100%,100%,100.0,Data Governance (Online Batch)
3,Stud0452,100%,100%,100%,100%,100%,100%,100%,100.0,Data Governance (Online Batch)
4,Stud0453,100%,100%,100%,100%,100%,100%,100%,100.0,Data Governance (Online Batch)
5,Stud0454,100%,100%,100%,100%,100%,100%,40%,91.0,Data Governance (Online Batch)
6,Stud0455,100%,100%,100%,100%,100%,100%,100%,100.0,Data Governance (Online Batch)
7,Stud0456,100%,100%,100%,100%,100%,100%,100%,100.0,Data Governance (Online Batch)
8,Stud0457,100%,100%,100%,100%,100%,100%,100%,100.0,Data Governance (Online Batch)
9,Stud0458,100%,100%,100%,100%,100%,100%,100%,100.0,Data Governance (Online Batch)


In [ ]:
# Data Quality Checks for assignments
# =================================

# 1. Completeness
missing_summary = assignments_df.isnull().sum()

# 2. Uniqueness
duplicate_rows = assignments_df.duplicated().sum()
duplicate_ids = assignments_df['Student_ID'].duplicated().sum()

# 3. Validity Checks
# Check Student_ID format
student_id_valid = assignments_df['Student_ID'].str.match(r'^Stud\d{4}$').all()

# Check week scores format (percentage strings)
week_cols = [f'week{i}' for i in range(1,8)]
percent_valid = assignments_df[week_cols].apply(lambda x: x.str.endswith('%')).all().all()

# Check final_score is numeric between 0-100
final_score_valid = ((assignments_df['final_score'] >= 0) & (assignments_df['final_score'] <= 100)).all()

# Check name column consistency
name_valid = assignments_df['name'].apply(lambda x: isinstance(x, str) and len(x) > 0).all()

# 4. Consistency
# Check if final_score matches weekly percentages
def calculate_expected_score(row):
    week_scores = [float(row[col].strip('%')) for col in week_cols]
    return sum(week_scores) / len(week_scores)

assignments_df['calculated_score'] = assignments_df.apply(calculate_expected_score, axis=1)
score_consistency = (abs(assignments_df['final_score'] - assignments_df['calculated_score']) < 1).all()

# 5. Anomaly Detection
# Check for suspicious patterns (all 100% scores)
all_perfect = (assignments_df[week_cols] == '100%').all(axis=1).sum()
perfect_final = (assignments_df['final_score'] == 100).sum()

# Data Quality Report
# ===================
assignments_quality_report = pd.DataFrame({
    "Dimension": [
        "Completeness",
        "Uniqueness",
        "Validity",
        "Consistency"
    ],
    "Result": [
        "Pass ✅" if missing_summary.sum() == 0 else "Fail ❌",
        "Pass ✅" if duplicate_rows == 0 and duplicate_ids == 0 else "Fail ❌",
        "Pass ✅" if all([student_id_valid, percent_valid, final_score_valid, name_valid]) else "Fail ❌",
        "Warning ⚠️" if not score_consistency else "Pass ✅"
    ],
    "Details": [
        "No missing values detected",
        f"{duplicate_rows} duplicate rows, {duplicate_ids} duplicate Student_IDs",
        f"Student IDs: {student_id_valid}, Week scores: {percent_valid}, Final scores: {final_score_valid}, Names: {name_valid}",
        f"{len(assignments_df[~score_consistency])} records with final_score ≠ weekly average" if not score_consistency else "Scores consistent"
    ]
})

assignments_quality_report

,Dimension,Result,Details
0,Completeness,Pass ✅,No missing values detected
1,Uniqueness,Pass ✅,"0 duplicate rows, 0 duplicate Student_IDs"
2,Validity,Pass ✅,"Student IDs: True, Week scores: True, Final sc..."
3,Consistency,Pass ✅,Scores consistent


# Cohort Assignment

In [ ]:
query = f"select * from Cohort_Assignment"
Cohort_Assignment_df = pd.read_sql(query, engine)
Cohort_Assignment_df.head()

,studentID,centerID,level,name,cohort,start_date,end_date,cohort_schedule,cohort_status,enrolment_confirmation,withdraw_reason1,withdraw_reason2,level_status_calculated,new_joiner,level_graduation_indicator,attendance_rate,quiz_complete,project_submit
0,Stud0427,Center06,L2,Python Fundamentals,C2,20-May-2024,24-May-2024,C2: 20-May-2024~24-May-2024,Withdraw,Confirmed,Left Company,None,1,Original,None,None,None,None
1,Stud0427,Center06,L2,ML Fundamentals,C3,6-May-2024,10-Jul-2024,C3: 6-May-2024~10-Jul-2024,Withdraw,Confirmed,Left Company,None,1,Original,None,None,None,None
2,Stud0427,Center06,L2,Generative AI Introduction,C4,15-Jul-2024,26-Jul-2024,C4: 15-Jul-2024~26-Jul-2024,Withdraw,Confirmed,Left Company,None,1,Original,None,None,None,None
3,Stud0427,Center06,L2,Large Language Model Introduction,C4,5-Aug-2024,4-Sep-2024,C4: 5-Aug-2024~4-Sep-2024,Withdraw,Confirmed,Left Company,None,4,Original,L2,None,None,None
4,Stud0427,Center06,L3,Data Preparation for GenAI,C4,20-Sep-2024,18-Oct-2024,C4: 20-Sep-2024~18-Oct-2024,Withdraw,Confirmed,Left Company,None,1,Original,None,None,None,None


In [ ]:
query = f"select * from cohort_courses"
cohort_courses_df = pd.read_sql(query, engine)
cohort_courses_df.head()

,id,name,weight,exam_id,client_id,cohort_id,published,created_at,description,resource_id,_airbyte_raw_id,_airbyte_extracted_at,_airbyte_meta
0,570,- Prompt Engineering (Cohort 7),"{""quizzes"": 20, ""projects"": 40, ""assignments"":...",143,client_1,297,0,2024-06-14 11:31:18.083810,,None,01e03b30-6f3a-4e89-8d44-55bdb63adcd0,2025-05-04 18:49:39.458,"{""changes"": []}"
1,640,- Computer Vision (Cohort 4),"{""quizzes"": 20, ""projects"": 40, ""assignments"":...",153,client_1,318,0,2025-01-02 19:24:26.505673,None,None,040a6a52-3233-494e-a374-fe51f16a4e32,2025-05-04 18:49:39.458,"{""changes"": []}"
2,633,- Data Modeling & Process (Cohort 5),"{""quizzes"": 20, ""projects"": 40, ""assignments"":...",151,client_1,321,0,2024-10-31 21:45:20.952750,None,None,062f79cf-1567-4e68-b1e0-f4ac421b3b9b,2025-05-04 18:49:39.458,"{""changes"": []}"
3,592,Applied BI,"{""quizzes"": 20, ""projects"": 40, ""assignments"":...",252,client_2,306,0,2024-08-01 17:26:25.032911,,None,0b7a0a42-edfe-4e9e-b70f-27d5c0bb1204,2025-05-04 18:49:39.458,"{""changes"": []}"
4,419,- NoCode AI,"{""quizzes"": 20, ""projects"": 40, ""assignments"":...",146,client_1,236,0,2024-01-15 10:39:46.369993,None,None,0bc97e84-5eaf-494c-a413-3e0a39acb590,2025-05-04 18:49:39.458,"{""changes"": []}"


In [ ]:
# Data Quality Checks for Cohort Assignment
# =========================================

# 1. Completeness
missing_summary = Cohort_Assignment_df.isnull().sum()

# 2. Uniqueness
duplicate_rows = Cohort_Assignment_df.duplicated().sum()
duplicate_ids = Cohort_Assignment_df['studentID'].duplicated().sum()

# 3. Validity Checks
# Check studentID format (e.g., Stud0427)
student_id_valid = Cohort_Assignment_df['studentID'].str.match(r'^Stud\d{4}$').all()

# Check date format
date_columns = ['start_date', 'end_date']
date_valid = True
for col in date_columns:
    try:
        pd.to_datetime(Cohort_Assignment_df[col], format="%d-%b-%Y")
    except:
        date_valid = False

# Check name non-empty strings
name_valid = Cohort_Assignment_df['name'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0).all()

# Check level is in expected set (e.g., L1, L2, L3, etc.)
level_valid = Cohort_Assignment_df['level'].str.match(r'^L\d$').all()

# 4. Consistency
# Check that cohort_schedule string contains the start and end dates
def cohort_schedule_check(row):
    return row['start_date'] in row['cohort_schedule'] and row['end_date'] in row['cohort_schedule']

cohort_schedule_consistency = Cohort_Assignment_df.apply(cohort_schedule_check, axis=1).all()

# 5. Anomaly Detection
# Check if same student is marked Withdraw across all rows
withdrawn_students = Cohort_Assignment_df.groupby('studentID')['cohort_status'].apply(lambda x: (x == 'Withdraw').all())
all_withdrawn = withdrawn_students.sum()

# Data Quality Report
# ===================
cohort_quality_report = pd.DataFrame({
    "Dimension": [
        "Completeness",
        "Uniqueness",
        "Validity",
        "Consistency"
    ],
    "Result": [
        "Pass ✅" if missing_summary.sum() == 0 else "Fail ❌",
        "Pass ✅" if duplicate_rows == 0 and duplicate_ids == 0 else "Fail ❌",
        "Pass ✅" if all([student_id_valid, date_valid, name_valid, level_valid]) else "Fail ❌",
        "Pass ✅" if cohort_schedule_consistency else "Warning ⚠️"
    ],
    "Details": [
        f"{missing_summary.sum()} missing values across {missing_summary[missing_summary > 0].count()} columns",
        f"{duplicate_rows} duplicate rows, {duplicate_ids} duplicate studentIDs",
        f"Student IDs: {student_id_valid}, Dates: {date_valid}, Names: {name_valid}, Levels: {level_valid}",
        "All cohort_schedule strings contain start and end dates" if cohort_schedule_consistency else "Some cohort_schedule values do not match date columns"
    ]
})

cohort_quality_report


,Dimension,Result,Details
0,Completeness,Fail ❌,7902 missing values across 9 columns
1,Uniqueness,Fail ❌,"0 duplicate rows, 1021 duplicate studentIDs"
2,Validity,Pass ✅,"Student IDs: True, Dates: True, Names: True, L..."
3,Consistency,Pass ✅,All cohort_schedule strings contain start and ...


In [ ]:
# Data Quality Checks for cohort_courses
# ===================================

# 1. Completeness
missing_summary = cohort_courses_df.isnull().sum()

# 2. Uniqueness
duplicate_rows = cohort_courses_df.duplicated().sum()
duplicate_ids = cohort_courses_df['id'].duplicated().sum()

# 3. Validity Checks
# Check ID columns are positive integers
id_valid = (cohort_courses_df[['id', 'exam_id', 'cohort_id']] > 0).all().all()

# Check client_id format
client_id_valid = cohort_courses_df['client_id'].str.startswith('client_').all()

# Check published is binary (0/1)
published_valid = cohort_courses_df['published'].isin([0, 1]).all()

# Check datetime formats
try:
    pd.to_datetime(cohort_courses_df['created_at'])
    datetime_valid = True
except:
    datetime_valid = False

# Improved weight column JSON structure validation
def is_valid_weight(weight):
    try:
        if pd.isna(weight):
            return False
        if isinstance(weight, str):
            import json
            weight = json.loads(weight.replace("'", '"'))  # Safer JSON parsing
        required_keys = ['quizzes', 'projects', 'assignments', 'attendances', 'soft_skills']
        if not all(k in weight for k in required_keys):
            return False
        if not isinstance(weight['attendances'], dict):
            return False
        attendance_keys = ['late', 'absence', 'excused', 'present']
        if not all(k in weight['attendances'] for k in attendance_keys):
            # Check for alternative structure with 'total' and 'participation'
            if 'total' not in weight['attendances'] or 'participation' not in weight['attendances']:
                return False
            if not all(k in weight['attendances']['participation'] for k in attendance_keys):
                return False
        return True
    except:
        return False

weight_valid = cohort_courses_df['weight'].apply(is_valid_weight).all()

# 4. Consistency
current_time = pd.Timestamp.now()
date_consistency = (pd.to_datetime(cohort_courses_df['created_at']) <= current_time).all()

# 5. Referential Integrity - Fixed query
# Get reference tables
cohorts_df = pd.read_sql("SELECT id FROM cohorts", engine)
exams_df = pd.read_sql("SELECT DISTINCT exam_id FROM cohort_courses WHERE exam_id IS NOT NULL", engine)

# Check cohort_ids exist in cohorts table
invalid_cohorts = cohort_courses_df[~cohort_courses_df['cohort_id'].isin(cohorts_df['id'])]
cohort_ref_valid = len(invalid_cohorts) == 0

# Check exam_ids exist in exams table
invalid_exams = cohort_courses_df[(cohort_courses_df['exam_id'].notna()) &
                                (~cohort_courses_df['exam_id'].isin(exams_df['exam_id']))]
exam_ref_valid = len(invalid_exams) == 0

# Data Quality Report
# ===================
cohort_courses_quality_report = pd.DataFrame({
    "Dimension": [
        "Completeness",
        "Uniqueness",
        "Validity",
        "Consistency",
        "Referential Integrity"
    ],
    "Result": [
        "Warning ⚠️" if missing_summary[['description', 'resource_id']].sum() > 0 else "Pass ✅",
        "Fail ❌" if duplicate_ids > 0 else "Pass ✅",
        "Fail ❌" if not weight_valid else "Pass ✅" if all([id_valid, client_id_valid, published_valid, datetime_valid]) else "Fail ❌",
        "Pass ✅",
        "Fail ❌" if not exam_ref_valid else "Pass ✅"
    ],
    "Details": [
        f"Nullable fields: description ({missing_summary['description']} nulls), resource_id ({missing_summary['resource_id']} nulls)",
        f"Duplicate IDs found: {duplicate_ids} (critical issue)",
        f"Weight JSON validation {'failed' if not weight_valid else 'passed'}. Other checks: IDs ({id_valid}), Client IDs ({client_id_valid}), Published ({published_valid}), Dates ({datetime_valid})",
        "All dates are chronologically valid",
        f"Referential integrity: Cohorts ({'valid' if cohort_ref_valid else f'{len(invalid_cohorts)} invalid'}), Exams ({'valid' if exam_ref_valid else f'{len(invalid_exams)} invalid'})"
    ]
})

cohort_courses_quality_report

,Dimension,Result,Details
0,Completeness,Warning ⚠️,"Nullable fields: description (204 nulls), reso..."
1,Uniqueness,Fail ❌,Duplicate IDs found: 280 (critical issue)
2,Validity,Pass ✅,Weight JSON validation passed. Other checks: I...
3,Consistency,Pass ✅,All dates are chronologically valid
4,Referential Integrity,Pass ✅,"Referential integrity: Cohorts (valid), Exams ..."


In [ ]:
cohort_courses_quality_report.to_csv('cohort_courses_quality_report.csv', index=False)

# cohort_course_enrollments

In [ ]:
query = f"SELECT * from cohort_course_enrollments"
cohort_course_enrollments_df = pd.read_sql(query, engine)
cohort_course_enrollments_df.head()

,id,status,exam_id,user_id,client_id,created_at,cohort_course_id,_airbyte_raw_id,_airbyte_extracted_at,_airbyte_meta
0,15620,enrolled,None,8396,client_1,2023-11-15 22:40:27.091870,391,0028ff59-c054-4cd4-9239-0e5b28aa71fd,2025-05-04 18:49:39.999,"{""changes"": []}"
1,18098,enrolled,None,9102,client_1,2024-03-04 11:57:43.002755,465,002f26cc-78ed-4e16-a00a-d1951e035da8,2025-05-04 18:49:39.999,"{""changes"": []}"
2,23560,enrolled,None,9791,client_1,2025-02-19 08:36:47.189379,642,0044f1b8-c7c7-4c98-b1a7-0fe9c54d5d06,2025-05-04 18:49:39.999,"{""changes"": []}"
3,23868,enrolled,None,8695,client_1,2025-04-08 14:04:22.152704,660,004b2e52-577c-40f0-9987-228b56dc5cea,2025-05-04 18:49:39.999,"{""changes"": []}"
4,23555,enrolled,None,9940,client_1,2025-02-19 08:36:46.956130,642,005dcc7f-19be-4f31-b565-bb2d0c54a54f,2025-05-04 18:49:39.999,"{""changes"": []}"


In [ ]:
# Data Quality Checks for cohort_course_enrollments (excluding metadata)
# =====================================================================

import pandas as pd
from datetime import datetime

# Remove Airbyte metadata columns
cols_to_analyze = ['id', 'status', 'exam_id', 'user_id', 'client_id', 'created_at', 'cohort_course_id']
enrollments_df = cohort_course_enrollments_df[cols_to_analyze]

# 1. Completeness
missing_summary = enrollments_df.isnull().sum()

# 2. Uniqueness
duplicate_rows = enrollments_df.duplicated().sum()
duplicate_ids = enrollments_df['id'].duplicated().sum()

# 3. Validity Checks
# Check ID formats (positive integers)
id_valid = (enrollments_df['id'] > 0).all()
user_id_valid = (enrollments_df['user_id'] > 0).all()
cohort_course_id_valid = (enrollments_df['cohort_course_id'] > 0).all()

# Check status values (expected: enrolled, completed, etc.)
status_values = enrollments_df['status'].unique()
status_valid = set(status_values).issubset({'enrolled', 'completed', 'dropped'})  # Add expected values

# Check timestamp format
created_at_valid = pd.to_datetime(enrollments_df['created_at'], errors='coerce').notna().all()

# Check client_id format (starts with 'client_' and has valid suffix)
client_id_valid = enrollments_df['client_id'].str.match(r'^client_[a-zA-Z0-9]+$').all()

# 4. Consistency
# Check if created_at dates are reasonable (not in future)
current_date = pd.to_datetime(datetime.now())
time_consistency = (pd.to_datetime(enrollments_df['created_at']) <= current_date).all()

# Check if all enrolled records have null exam_id (if expected)
exam_id_consistency = True
if 'enrolled' in status_values:
    enrolled_with_exam = ((enrollments_df['status'] == 'enrolled') &
                         (enrollments_df['exam_id'].notnull())).sum()
    exam_id_consistency = enrolled_with_exam == 0

# 5. Anomaly Detection
# Check for duplicate user_id + cohort_course_id combinations
user_cohort_duplicates = enrollments_df.duplicated(subset=['user_id', 'cohort_course_id'], keep=False).sum()

# Check for users enrolled in same course multiple times
multiple_enrollments = enrollments_df.groupby(['user_id', 'cohort_course_id'])['id'].count()
multiple_enrollments = multiple_enrollments[multiple_enrollments > 1].count()

# Data Quality Report
# ===================
enrollments_quality_report = pd.DataFrame({
    "Dimension": [
        "Completeness",
        "Uniqueness",
        "Validity",
        "Consistency",
        "Anomalies"
    ],
    "Result": [
        "Warning ⚠️" if missing_summary['exam_id'] > 0 else "Pass ✅",
        "Pass ✅" if duplicate_rows == 0 and duplicate_ids == 0 else "Fail ❌",
        "Pass ✅" if all([id_valid, user_id_valid, cohort_course_id_valid,
                         status_valid, created_at_valid, client_id_valid]) else "Fail ❌",
        "Pass ✅" if time_consistency and exam_id_consistency else "Warning ⚠️",
        "Warning ⚠️" if user_cohort_duplicates > 0 else "Pass ✅"
    ],
    "Details": [
        f"Missing values: exam_id ({missing_summary['exam_id']}), others complete",
        f"{duplicate_rows} duplicate rows, {duplicate_ids} duplicate IDs",
        f"IDs valid: {id_valid}, Statuses: {status_values}, Timestamps valid: {created_at_valid}",
        f"Time consistency: {time_consistency}, Exam ID rules: {exam_id_consistency}",
        f"{user_cohort_duplicates} user+cohort duplicates found"
    ]
})

enrollments_quality_report

,Dimension,Result,Details
0,Completeness,Warning ⚠️,"Missing values: exam_id (11795), others complete"
1,Uniqueness,Fail ❌,"7863 duplicate rows, 7863 duplicate IDs"
2,Validity,Fail ❌,"IDs valid: True, Statuses: ['enrolled' 'not-en..."
3,Consistency,Pass ✅,"Time consistency: True, Exam ID rules: True"
4,Anomalies,Warning ⚠️,11795 user+cohort duplicates found


# Quizzes

In [ ]:
query = f"SELECT * from Quizzes"
Quizzes_df = pd.read_sql(query, engine)
Quizzes_df.head(50)

,Student_ID,Quiz1,Quiz2,Quiz3,Quiz4,Quiz5,Quiz6,Quiz7,Quiz8,name
0,Stud0449,100.0,95.0,100.0,100.00,100.00,NaN,NaN,NaN,Data Governance (Online Batch)
1,Stud0450,100.0,90.0,100.0,100.00,100.00,NaN,NaN,NaN,Data Governance (Online Batch)
2,Stud0451,100.0,100.0,100.0,100.00,100.00,NaN,NaN,NaN,Data Governance (Online Batch)
3,Stud0452,80.0,95.0,100.0,100.00,100.00,NaN,NaN,NaN,Data Governance (Online Batch)
4,Stud0453,90.0,95.0,100.0,100.00,100.00,NaN,NaN,NaN,Data Governance (Online Batch)
5,Stud0454,100.0,90.0,100.0,100.00,100.00,NaN,NaN,NaN,Data Governance (Online Batch)
6,Stud0455,95.0,85.0,100.0,100.00,100.00,NaN,NaN,NaN,Data Governance (Online Batch)
7,Stud0456,95.0,90.0,100.0,100.00,100.00,NaN,NaN,NaN,Data Governance (Online Batch)
8,Stud0457,100.0,100.0,100.0,100.00,100.00,NaN,NaN,NaN,Data Governance (Online Batch)
9,Stud0458,95.0,95.0,100.0,100.00,100.00,NaN,NaN,NaN,Data Governance (Online Batch)


In [ ]:
# Data Quality Checks for Quizzes
# ==============================

# 1. Completeness
missing_summary = Quizzes_df.isnull().sum()

# 2. Uniqueness
duplicate_rows = Quizzes_df.duplicated().sum()
duplicate_ids = Quizzes_df['Student_ID'].duplicated().sum()

# 3. Validity Checks
# Check Student_ID format (e.g., Stud0427)
student_id_valid = Quizzes_df['Student_ID'].str.match(r'^Stud\d{4}$').all()

# Check quiz columns are non-null (assuming all quizzes should have data)
quiz_columns = ['Quiz1', 'Quiz2', 'Quiz3', 'Quiz4', 'Quiz5', 'Quiz6', 'Quiz7', 'Quiz8']
quiz_valid = Quizzes_df[quiz_columns].notnull().all().all()

# Check name non-empty strings
name_valid = Quizzes_df['name'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0).all()

# 4. Consistency
# Check if all quizzes have consistent values (if quizzes should not have zero values or negative)
quiz_value_consistency = (Quizzes_df[quiz_columns] >= 0).all().all()

# 5. Anomaly Detection
# Check for any duplicate student IDs
duplicate_student_ids = Quizzes_df['Student_ID'].duplicated().sum()

# Data Quality Report
# ===================
quizzes_quality_report = pd.DataFrame({
    "Dimension": [
        "Completeness",
        "Uniqueness",
        "Validity",
        "Consistency",
        "Anomalies"
    ],
    "Result": [
        "Pass ✅" if missing_summary.sum() == 0 else "Fail ❌",
        "Pass ✅" if duplicate_rows == 0 and duplicate_ids == 0 else "Fail ❌",
        "Pass ✅" if all([student_id_valid, quiz_valid, name_valid]) else "Fail ❌",
        "Pass ✅" if quiz_value_consistency else "Fail ❌",
        "Pass ✅" if duplicate_student_ids == 0 else "Warning ⚠️"
    ],
    "Details": [
        f"{missing_summary.sum()} missing values across {missing_summary[missing_summary > 0].count()} columns",
        f"{duplicate_rows} duplicate rows, {duplicate_ids} duplicate student IDs",
        f"Student IDs: {student_id_valid}, Quizzes valid: {quiz_valid}, Names valid: {name_valid}",
        "All quiz values are non-negative" if quiz_value_consistency else "Some quiz values are negative or zero",
        f"{duplicate_student_ids} duplicate student IDs found"
    ]
})

quizzes_quality_report


,Dimension,Result,Details
0,Completeness,Fail ❌,105 missing values across 3 columns
1,Uniqueness,Pass ✅,"0 duplicate rows, 0 duplicate student IDs"
2,Validity,Fail ❌,"Student IDs: True, Quizzes valid: False, Names..."
3,Consistency,Fail ❌,Some quiz values are negative or zero
4,Anomalies,Pass ✅,0 duplicate student IDs found
